# E02：Metropolis 采样一维谐振子（高斯）

目标分布：
\[
\pi(x) \propto \exp\left(-\beta \frac{x^2}{2}\right)
\]
解析结果：均值 0、方差 \(1/\beta\)。

你将：
1. 调用 `exercises/src/e02_metropolis_harmonic.py` 的采样器生成样本  
2. 画出直方图并与解析高斯对比  
3. 调整 step_size 观察接受率/自相关的变化（可选）

> 自动检查：`pytest -q` 会检查你的采样结果是否合理。


In [ ]:
import os
import sys

# Add statphys_urban_learning to sys.path for local imports
curr = os.path.abspath('')
while curr != os.path.dirname(curr):
    if 'statphys_urban_learning' in os.listdir(curr):
        target = os.path.join(curr, 'statphys_urban_learning')
        if target not in sys.path:
            sys.path.insert(0, target)
        break
    curr = os.path.dirname(curr)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from exercises.src.e02_metropolis_harmonic import MetropolisConfig, metropolis_harmonic, analytic_mean_var

from exercises.src.mcmc_diagnostics import integrated_autocorr_time


In [ ]:
cfg = MetropolisConfig(
    n_steps=80_000,
    beta=2.0,
    step_size=1.2,
    x0=5.0,
    burn_in=10_000,
    thin=20,
    seed=42,
)
samples = metropolis_harmonic(cfg)

m_hat = samples.mean()
v_hat = samples.var()
m_true, v_true = analytic_mean_var(cfg.beta)

m_hat, v_hat, m_true, v_true, len(samples)

In [ ]:
# 画直方图与解析曲线
xs = np.linspace(samples.min(), samples.max(), 400)
# 解析高斯（未归一化也行，但我们这里写归一化形式）
sigma2 = v_true
pdf = 1.0/np.sqrt(2*np.pi*sigma2) * np.exp(-(xs-m_true)**2/(2*sigma2))

plt.figure()
plt.hist(samples, bins=60, density=True)
plt.plot(xs, pdf)
plt.title("Metropolis samples vs analytic Gaussian")
plt.xlabel("x"); plt.ylabel("density")
plt.show()


## 思考题（可选）
1. 把 `step_size` 改得很小/很大，会发生什么？为什么？
2. 用你自己的方法估计有效样本量（ESS）或自相关时间。
3. 把能量函数换成双阱势 \(E(x)=\frac{1}{4}(x^2-1)^2\)，观察多峰采样的困难。


In [ ]:
# --- Minimal MCMC diagnostics (tau_int and ESS) ---
diag = integrated_autocorr_time(samples, max_lag=500)

se_mean = samples.std(ddof=1) / np.sqrt(diag.ess)

diag, se_mean
